In [1]:
import cv2
import numpy as np

In [2]:
def show(img, offset=1):
    shape = img.shape
#     offset = 1
    s = 0
    if shape[0] > 1000:
        s = int(10/(int(str(shape[0])[0])+offset))*0.1
    elif shape[1] > 1000:
        s = int(10/(int(str(shape[1])[0])+offset))*0.1
    if s:
        img = cv2.resize(img, None, fx=s, fy=s)
    cv2.imshow("Img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
from skimage import io
url = 'https://www.localmt.in/wp-content/uploads/2022/05/FullSizeRender11-350x350.jpg'
image = io.imread(url)

HTTPError: HTTP Error 403: Forbidden

In [8]:
from urllib.request import urlopen
def url_to_image(url, readFlag=cv2.IMREAD_COLOR):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    resp = urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, readFlag)

    # return the image
    return image

In [10]:
import imutils
imutils.url_to_image(url)

HTTPError: HTTP Error 403: Forbidden

In [25]:
import requests
from PIL import Image
from io import BytesIO

In [21]:
r = requests.get(url)

In [26]:
img = Image.open(BytesIO(requests.get(url).content))

In [33]:
i = np.array(img)
i = cv2.cvtColor(i, cv2.COLOR_RGB2BGR)

In [35]:
show(i)